In [ ]:
import scipy.io as sio
import os
import numpy as np
import tensorflow as tf
datapath = r'..\data'
dataset = 'NIPS_INSECT'
x = sio.loadmat(os.path.join(datapath, dataset, 'res101.mat'))
x2 = sio.loadmat(os.path.join(datapath, dataset, 'att_splits.mat')) 
barcodes=x['nucleotides_aligned'].T
species=x['labels']
train_loc=x2['trainval_loc']

In [ ]:
# Number of training samples and entire data
N = len(barcodes)
print(len(train_loc[0]), N)

In [ ]:
# Reading barcodes and labels into python list
bcodes=[]
labels=[]
for i in range(N):
    if len(barcodes[i][0][0])>0:
        bcodes.append(barcodes[i][0][0])
        labels.append(species[i][0])
    

In [ ]:
print('Sample barcode: %s' % bcodes[0])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Converting base pairs, ACTG, into numbers
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(bcodes)
sequence_of_int = tokenizer.texts_to_sequences(bcodes)

In [ ]:
# For INSECT dataset, majority of barcodes have a length of 658
cnt = 0
ll = np.zeros((N, 1))
for i in range(N):
    ll[i] = len(sequence_of_int[i])
    if ll[i]==658:
        #print(i)
        cnt += 1
print(cnt, np.median(ll))

In [ ]:
import numpy as np

# Calculating sequence count for each species
cl = len(np.unique(species))
seq_len=np.zeros((cl))
seq_cnt=np.zeros((cl))
for i in range(N):
    k=labels[i]-1  # Accounting for Matlab labeling
    seq_cnt[k]+=1
    #seq_len[k]+=len(sequence_of_int[k])
    
print('# classes: %d' % cl)

In [ ]:
# Converting all the data into one-hot encoding. Note that this data is not used during training. 
# We are getting it ready for the prediction time to get the final DNA embeddings after training is done
sl = 658
allX=np.zeros((N,sl,5))
for i in range(N):
    Nt=len(sequence_of_int[i])

    for j in range(sl):
        if(len(sequence_of_int[i])>j):
            k=sequence_of_int[i][j]-1
            if(k>4):
                k=4
            allX[i][j][k]=1.0
            

In [ ]:
# Initialize the training matrix and labels
trainX=np.zeros((N,sl,5))
trainY=np.zeros((N,cl))
labelY=np.zeros(N)

In [ ]:
# Here is where we cap the class sample size to 50 for a balanced training set
Nc=-1
clas_cnt=np.zeros((cl))
for i in range(N):
    Nt=len(sequence_of_int[i])
    k=labels[i]-1
    clas_cnt[k]+=1
    itl=i+1
    if(seq_cnt[k]>=10 and clas_cnt[k]<=50 and itl in train_loc[0]): # Note that samples from training set are only used 
        Nc=Nc+1
        for j in range(sl):
            if(len(sequence_of_int[i])>j):
                k=sequence_of_int[i][j]-1
                if(k>4):
                    k=4
                trainX[Nc][j][k]=1.0
            
        k=labels[i]-1
        trainY[Nc][k]=1.0
        labelY[Nc]=k

In [ ]:
print('Final training size: %d' % Nc)

In [ ]:
# selecting the balanced training set
trainX=trainX[0:Nc]
trainY=trainY[0:Nc]
labelY=labelY[0:Nc]
print(trainX.shape, trainY.shape, labelY.shape)

In [ ]:
# To make sure the training data does not include any unseen class nucleotides
label_us = species[x2['test_unseen_loc'][0]-1]
np.intersect1d(np.unique(labelY), np.unique(label_us)-1)

In [ ]:
# Cleaning empty classes
idx = np.argwhere(np.all(trainY[..., :] == 0, axis=0))
trainY = np.delete(trainY, idx, axis=1)

print(len(idx), min(np.sum(trainY,axis=0)))

In [ ]:
print(trainY.shape)
print(labelY.shape)

In [ ]:
# Expanding the training set shape for CNN 
trainX=np.expand_dims(trainX, axis=3)
allX=np.expand_dims(allX, axis=3)
print(trainX.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the training set into train and validation
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

In [ ]:
print(y_train.shape)

### Classic CNN Model

In [ ]:
from tensorflow.keras import datasets, layers, models, optimizers, callbacks

In [ ]:
# CNN model architecture
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(sl, 5,1),padding="SAME"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Conv2D(32, (3,3), activation='relu',padding="SAME"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Conv2D(16, (3,3), activation='relu',padding="SAME"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3,1)))
# model.add(layers.Conv2D(16, (3,3), activation='relu',padding="SAME"))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(500, activation='tanh'))
model.add(layers.Dense(y_train.shape[1]))

In [ ]:
model.summary()

In [ ]:
# Step-decay learning rate scheduler
def step_decay(epoch):
   initial_lrate = 0.001
   drop = 0.5
   epochs_drop = 2.0
   lrate = initial_lrate * np.power(drop, np.floor((1+epoch)/epochs_drop))
   return lrate

class LossHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
       self.losses = []
       self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
       self.losses.append(logs.get('loss'))
       self.lr.append(step_decay(len(self.losses)))
        
loss_history = LossHistory()
lrate = callbacks.LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

In [ ]:
from tensorflow.keras.metrics import top_k_categorical_accuracy
#opt = tf.keras.optimizers.SGD(momentum=0.9, nesterov=True)
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy','top_k_categorical_accuracy'])

# Validation time
history = model.fit(X_train, y_train, epochs=5, batch_size = 32, validation_data=(X_test, y_test), callbacks=callbacks_list, verbose=1)

# Final Test time
#history = model.fit(trainX, trainY, epochs=5, callbacks=callbacks_list)

In [ ]:
print('Learning rates through epochs:', loss_history.lr)

In [ ]:
from tensorflow.keras.models import Model

# Getting the DNA embeddings of all data from the last dense layer
layer_name = 'dense'
model2= Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
dna_embeddings=model2.predict(allX)
print(dna_embeddings.shape)

In [ ]:
# saving the results into csv file to be later read into matlab
np.savetxt("nips_cnn_embeddings_500_5e_adam_aligned.csv", dna_embeddings, delimiter=",")